In [1]:
from scraper import pipeline_runner
from feature_vectors import *
import pandas as pd

#Small Dataframe

###Only using CountVectorizer, no additional regex feature vectors

In [2]:
df_50x10 = pd.read_pickle('scraper_50x10.pkl')

In [3]:
df_mnb = pipeline_runner(df_50x10, 'Multinomial')
df_knn = pipeline_runner(df_50x10, 'KNeighbors')
df_forest = pipeline_runner(df_50x10, 'Forest')

In [6]:
print("df_mnb: {}".format(df_mnb))
print("df_knn: {}".format(df_knn))
print("df_forest: {}".format(df_forest))

df_mnb: (0.89317507418397624, 0.51333333333333331)
df_knn: (0.49406528189910981, 0.31777777777777777)
df_forest: (0.99183976261127593, 0.62666666666666671)


###Getting low scores. Random Forest is the most accurate with 0.62 mean score. 

#Larger Dataframe

###Still no additional feature vectorizers

In [2]:
#see scraper.py for how the 500x100 dataframe was pulled from RosettaCode. 
df_500x100 = pd.read_pickle('scraper_500x100.pkl')

In [8]:
df_mnb_500 = pipeline_runner(df_500x100, 'Multinomial')
df_knn_500 = pipeline_runner(df_500x100, 'KNeighbors')
df_forest_500 = pipeline_runner(df_500x100, 'Forest')

In [9]:
print("df_mnb_500: {}".format(df_mnb_500))
print("df_knn_500: {}".format(df_knn_500))
print("df_forest_500: {}".format(df_forest_500))

df_mnb_500: (0.80047166883882581, 0.62267311988086371)
df_knn_500: (0.60628064295910133, 0.40171258376768426)
df_forest_500: (0.98907714267982372, 0.72040208488458679)


##Random Forest, again, produces the best mean accuracy. 

#Adding additional feature vectors to improve score accuracy

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [4]:
X = df_500x100.loc[:, 1]
y = df_500x100.loc[:, 0]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [9]:
language_featurizer = make_union(CountVectorizer(),
                                     FunctionFeaturizer(longest_run_of_capital_letters_feature,
                                                        longest_run_of_character_feature,
                                                        percent_character_combinations,
                                                        percent_character_feature,
                                                        binary_character_combinations))

In [10]:
language_featurizer.fit_transform(X)

<21485x67251 sparse matrix of type '<class 'numpy.float64'>'
	with 833585 stored elements in Compressed Sparse Row format>

In [11]:
pipe = make_pipeline(language_featurizer, RandomForestClassifier())

In [13]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
 ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [14]:
pipe.score(X_test, y_test)

0.76954579300074455

The test mean accuracy is 0.76 using only the data pulled from RosettaCode (not using the test data from the file).

###Score accuracy imporved 0.05 by adding additional feature vectors. 